In [1]:
#code to record X mins of audio, overlay gunshot clips on top of it, run inference on it with both the 1D and 2D
#tflite models, and output precision, recall, etc, in order to evaluate the performance of models
    #can easily jump to overlaying clips/inference if you already have a recording
    
    
import pyaudio
import librosa
import wave
import numpy as np
from threading import Thread
from array import array
from scipy.io import wavfile
import soundfile as sf
import scipy.signal
from queue import Queue
import time
import os
import tensorflow as tf
import tensorflow.keras as keras

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/librosa/display.py:32: MatplotlibDeprecationWarning: 
The examples.directory rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2. In the future, examples will be found relative to the 'datapath' directory.
  mpl.rcParams.update(**_matplotlibrc)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/_collections_abc.py:849: MatplotlibDeprecationWarning: 
The examples.directory rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2. In the future, examples will be found relative to the 'datapath' directory.
  self[key] = value
/Library/Frameworks/Python.framework/Vers

## Variables and Functions

In [26]:
#variables
audio_format = pyaudio.paInt16
audio_rate = 44100
audio_channels = 1
audio_device_index = 2
audio_frames_per_buffer = 4410
audio_sample_duration = 2
sound_data = np.zeros(0, dtype = "int16")
sound_normalization_threshold = 10 ** (-1.0 / 20)
max_audio_frame_int_value = 2 ** 15 - 1


#how long to record audio for
recording_length = 10
#how many gunshot clips to put on top of your audio
n_gunshot_files_to_use = 100

#booleans for which model to use
USING_2D_RAW_SPECTROGRAM_MODEL = True
USING_1D_TIME_SERIES_MODEL = False

#directory to save files to
#for my laptop
files_directory = "/Users/laurenogden/Downloads/"
#relative path to directory on github
#files_directory = "../recordings/"

#directory of gunshot files to put on top of data (2s or less long files)
gunshot_directory = files_directory + "Original Gunshot Trimmed/"

audio_analysis_queue = Queue()

In [3]:
#normalization function
def normalize(sound_data):
    normalization_factor = float(sound_normalization_threshold * max_audio_frame_int_value) / max(abs(i) for i in sound_data)
    
    # Averages the volume out
    r = array('h')
    for datum in sound_data:
        r.append(int(datum * normalization_factor))
    return np.array(r, dtype = np.int16)

In [4]:
#function to save a wav file
def create_wav_file(microphone_data, name):
    librosa.output.write_wav(files_directory + name + ".wav", microphone_data, 22050)

In [5]:
#prints the times of each sound (mins : secs)
def time_in_mins(index):
    secs = index/22050
    return str(int(secs//60)) + ":" + str(secs%60)

In [6]:
#makes a spectrogram for 2d CNN
def convert_to_spectrogram(data, sample_rate):
    return np.array(librosa.feature.melspectrogram(y = data, sr = sample_rate), dtype = "float32")

In [7]:
#callback function for pyaudio strean
def callback(in_data, frame_count, time_info, status):
    global sound_data
    sound_buffer = np.frombuffer(in_data, dtype = "int16")
    sound_data = np.append(sound_data, sound_buffer)
    if len(sound_data) >= 88200:
        audio_analysis_queue.put(sound_data)
        #empty out sound_data
        sound_data = np.zeros(0, dtype = "int16")

    return (sound_buffer, pyaudio.paContinue)

## Record X mins of audio

In [ ]:
#open pyaudio stream
pa = pyaudio.PyAudio()
stream = pa.open(format = audio_format,
                 rate = audio_rate,
                 channels = audio_channels,
                 input_device_index = audio_device_index,
                 frames_per_buffer = audio_frames_per_buffer,
                 input = True,
                 stream_callback = callback)

# Starts the callback thread
stream.start_stream()

#get first bit of mic data from the stream
mic_data = audio_analysis_queue.get()
mod_mic_data = librosa.resample(y = mic_data, orig_sr = audio_rate, target_sr = 22050)
mod_mic_data = normalize(mod_mic_data)
mic_data = mod_mic_data
#create_wav_file(mod_mic_data, "-1_" + str(time.time()))

#get 10 mins of audio data (300 2s clips)
for i in range(0, recording_length*30 - 1):
    #print(time.ctime(time.time()))
    new_data = np.array(audio_analysis_queue.get(), dtype = "int16")
    mod_new_data = librosa.resample(y = new_data, orig_sr = audio_rate, target_sr = 22050)
    mod_mic_data = normalize(mod_new_data)
    #create_wav_file(mod_new_data, str(i) + "_" + str(time.time()))
    mic_data = np.append(mic_data, mod_new_data)

#save the clip
createwav_file(mic_data, str(recording_length) + "_mins_background")

### Mark loud noises in noise clip

In [ ]:
#mark locations of loud noises in the clips (taps/claps/etc)

#sort the data, figure out the threshold
sorted_data = np.sort(mic_data)
threshold = sorted_data[len(sorted_data) - int(len(sorted_data)*0.001)]

#find all values above that threshold
above_threshold = []
for i in range(0, len(mic_data)):
    if mic_data[i] > threshold:
        above_threshold.append(i)

#separate out individual sounds from that whole chunk
distinct_sounds = []
distinct_sounds.append(above_threshold[0])
for i in range(1, len(above_threshold)):
    #if within 5ms of each other, assume from same shot
    if above_threshold[i] - above_threshold[i-1] > 0.05*22050:
        distinct_sounds.append(above_threshold[i])

#times relative to beginning of the saved clip
print("There were " + str(len(distinct_sounds)) + " distinct loud sounds detected")
distinct_times = []
for i in distinct_sounds:
    distinct_times.append(i/22050)
    
#save them in a txt file for future possible use
loud_noises_file = open(files_directory + "background_loud_noises_indices.txt", "w")
for i in distinct_sounds:
    loud_noises_file.write(str(i)+"\n")
loud_noises_file.close()
    

## Overlay gunshot clips onto recorded noise clip

In [27]:
#load back the X min clip saved above (or whenever, idc)
noise, rate = librosa.load(files_directory + "10mins_lab.wav")
#noise, rate = librosa.load(files_directory + str(recording_length) + "_mins_background.wav")

#list files in a gunshot audio directory
gunshot_files = os.listdir(gunshot_directory)

In [10]:
# make a numpy array the size of your clip, fill it with zeros
all_gunshots = np.zeros(len(noise), dtype = "float32")

locs = []
locs_of_actual_shots = []
files_used = []
#add gunshots to that 0 numpy array
for i in range(0, n_gunshot_files_to_use):
    #pick a random gunshot file
    file = gunshot_files[np.random.randint(0, len(gunshot_files))]
    #avoid using the exact same gunshot twice, or attempting to load a non-wav file in the folder
    while file in files_used and ".wav" not in file:
        file = gunshot_files[np.random.randint(0, len(gunshot_files))]
    files_used.append(file)
    #load the file
    gunshot, sr = librosa.load(gunshot_directory + file)
    
    
    #find the location(s) of the actual gunshot(s) in that file
    #sort the data, figure out the threshold
    sorted_gunshot_data = np.sort(gunshot)
    threshold = sorted_gunshot_data[len(sorted_gunshot_data) - int(len(sorted_gunshot_data)*0.001)]
    #find all values above that threshold
    above_threshold = []
    for i in range(0, len(gunshot)):
        if gunshot[i] > threshold:
            above_threshold.append(i)
    #separate out individual sounds from that whole chunk
    distinct_shots = []
    distinct_shots.append(above_threshold[0])
    for i in range(1, len(above_threshold)):
        #if within 5ms of each other, assume from same shot
        if above_threshold[i] - above_threshold[i-1] > 0.05*22050:
            distinct_shots.append(above_threshold[i])
    
    
    #pick a random location to put it at 
    loc = np.random.randint(0, len(noise) - 44100)
    #to avoid putting two gunshots at the exact same place in the clip
        #to be improved potentially to avoid overlapping gunshots??
    while loc in locs:
        loc = np.random.randint(0, len(noise))
    locs.append(loc)
    #append location(s) of the actual gunshot(s) in the entire clip
    for i in distinct_shots:
        locs_of_actual_shots.append(loc+i)
    
    #print("putting " + file + " at location " + str(loc) + ", time= " + time_in_mins(loc))
    
    #place the data at that location
    for j in range(loc, loc+len(gunshot)):
        all_gunshots[j] = all_gunshots[j] + gunshot[j-loc]
    
#combine the noise clip with the gunshot array and save it
overlaid = noise * 0.7 + all_gunshots * 1.2
overlaid_normed = normalize(overlaid)
create_wav_file(overlaid_normed, str(recording_length) + "_mins_overlaid")

### Save locations of gunshots in new overlaid clip

In [30]:
#sort the list of locations of actual gunshots
print("Added " + str(len(locs_of_actual_shots)) + " gunshots")
locs_of_actual_shots.sort()
'''
print("Times of the gunshots in the recording: ")
for i in locs_of_actual_shots:
    print("location = " + str(i) + " at time = " + str(time_in_mins(i)))
'''
    
#figure out what two second clips contain gunshots
clips_w_guns = []
for i in range(0, len(noise), 44100):
    for j in locs_of_actual_shots:
        if j in range(i, i+44100):
            if i not in clips_w_guns:
                clips_w_guns.append(i)

print("There are " + str(len(clips_w_guns)) + " 2s clips in our " + str(len(noise)/22050/60) 
      + " min recording containing gunshots")

Added 151 gunshots
There are 86 2s clips in our 10.0 min recording containing gunshots


In [31]:
#save all of the clips with guns for evaluation purposes if you feel like it
'''
for i in clips_w_guns:
    #print(time_in_mins(i))
    create_wav_file(overlaid_normed[i: i+44100], "contains_gunshot_" + time_in_mins(i))
'''
    
#save the indices of the clips with guns in a textfile for later use
locations_file = open(files_directory + "gunshot_clip_indices.txt", "w")
for i in clips_w_guns:
    locations_file.write(str(i)+"\n")
locations_file.close()

## Run Models on Clips

In [15]:
#load back in the clip with the gunshots
audio, sr = librosa.load(files_directory + "10mins_overlaid.wav")
#audio, sr = librosa.load(files_directory + str(recording_length) + "_mins_overlaid.wav")

#load back the indices of clips that contain gunshots from the txt file
locations_file = open(files_directory + "gunshot_clip_indices.txt", "r")
clips_w_guns = locations_file.readlines()
locations_file.close()
#turn strings to ints
clips_w_guns = list(map(int, clips_w_guns))
#print(clips_w_guns)

### Load Models

#### 1D TFLite

In [33]:
#load 1D tflite model
interpreter_1D = tf.lite.Interpreter(model_path="../models/gunshot_sound_model.tflite")
interpreter_1D.allocate_tensors()
# Get input and output tensors.
input_details_1D = interpreter_1D.get_input_details()
output_details_1D = interpreter_1D.get_output_details()
input_shape_1D = input_details_1D[0]['shape']


#### 2D TFLite

In [ ]:
#load 2D tflite model
interpreter_2D = tf.lite.Interpreter(model_path="../models/spectro_no_variables.tflite")
interpreter_2D.allocate_tensors()
# Get input and output tensors.
input_details_2D = interpreter_2D.get_input_details()
output_details_2D = interpreter_2D.get_output_details()
input_shape_2D = input_details_2D[0]['shape']

### Inference

#### 1D TFLite

In [34]:
positives_1D = []

#pass two second slices into the model to predict
for i in range (0, len(audio), 44100):
    audio_slice = audio[i:i+44100]
    
    #1D reshaping
    reshaped_audio_slice_1D = audio_slice.reshape(-1, 44100, 1)
    
    # https://stackoverflow.com/questions/54006031/tf-lite-model-test-fails-with-run-time-error
        
    #1D predictions
    input_tensor_1D = tf.convert_to_tensor(reshaped_audio_slice_1D, np.float32)
    interpreter_1D.set_tensor(input_details_1D[0]["index"], reshaped_audio_slice_1D)
    interpreter_1D.invoke()
    probabilities_1D = interpreter_1D.get_tensor(output_details_1D[0]["index"])
    if probabilities_1D[0][1] >= 0.95:
        #create_wav_file(audio_slice, str(i))
        positives_1D.append(i)    


#### 2D TFLite

In [ ]:
positives_2D = []

#pass two second slices into the model to predict
for i in range (0, len(audio), 44100):
    audio_slice = audio[i:i+44100]
    
    #2D reshaping
    reshaped_audio_slice_2D = convert_to_spectrogram(data = audio_slice, sample_rate = 22050)
    reshaped_audio_slice_2D = reshaped_audio_slice_2D.reshape(-1, 128, 87, 1)
        
    # https://stackoverflow.com/questions/54006031/tf-lite-model-test-fails-with-run-time-error
        
    #2D predictions
    input_tensor_2D = tf.convert_to_tensor(reshaped_audio_slice_2D, np.float32)
    interpreter_2D.set_tensor(input_details_2D[0]["index"], reshaped_audio_slice_2D)
    interpreter_2D.invoke()
    probabilities_2D = interpreter_2D.get_tensor(output_details_2D[0]["index"])
    if probabilities_2D[0][1] >= 0.95:
        #create_wav_file(audio_slice, str(i))
        positives_2D.append(i) 
    

### Separate TP, FP, TN, FN and Output Metrics

### 1D Model

In [35]:
false_positives_1D = []
true_positives_1D = []
false_negatives_1D = []
true_negatives_1D = []

#figure out all true positives
for i in positives_1D:
    if i in clips_w_guns:
        true_positives_1D.append(i)
        create_wav_file(audio[i: i+44100], "1D_true_positive_" + str(i))

#figure out all false positives
for i in positives_1D:
    if i not in true_positives_1D:
        false_positives_1D.append(i)
        create_wav_file(audio[i:i+44100], "1D_false_positive_" + str(i))
        
#figure out all false negatives
for i in clips_w_guns:
    if i not in positives_1D:
        #missed_gunshots.append(i)
        #slice_location = i//44100
        false_negatives_1D.append(i)
        create_wav_file(audio[i:i+44100], "1D_false_negative_" + str(i))

In [36]:
total = recording_length*30
total_pos_1D = len(positives_1D)
total_neg_1D = total - total_pos_1D
TP_1D = len(true_positives_1D)
FP_1D = len(false_positives_1D)
FN_1D = len(false_negatives_1D)
TN_1D = total_neg_1D - FN_1D 

print("1D Model Metrics:")
print("Total # of clips classified as gunshots: " + str(total_pos_1D))
print("True Positives: " + str(TP_1D))
print("False Positives: " + str(FP_1D))
print("False Negatives: " + str(FN_1D))
print("True Negatives: " + str(TN_1D))

precision_1D = TP_1D / (TP_1D + FP_1D)
recall_1D = TP_1D / (TP_1D + FN_1D)
accuracy_1D = (TP_1D + TN_1D) / total
print("Precision: " + str(precision_1D))
print("Recall: " + str(recall_1D))
print("Accuracy: " + str(accuracy_1D))

1D Model Metrics:
Total # of clips classified as gunshots: 4
True Positives: 4
False Positives: 0
False Negatives: 82
True Negatives: 214
Precision: 1.0
Recall: 0.046511627906976744
Accuracy: 0.7266666666666667


### 2D Model

In [37]:
false_positives_2D = []
true_positives_2D = []
false_negatives_2D = []
true_negatives_2D = []

#figure out all true positives
for i in positives_2D:
    if i in clips_w_guns:
        true_positives_2D.append(i)
        create_wav_file(audio[i: i+44100], "2D_true_positive_" + str(i))

#figure out all false positives
for i in positives_2D:
    if i not in true_positives_2D:
        false_positives_2D.append(i)
        create_wav_file(audio[i:i+44100], "2D_false_positive_" + str(i))
        
#figure out all false negatives
for i in clips_w_guns:
    if i not in positives_2D:
        #missed_gunshots.append(i)
        #slice_location = i//44100
        false_negatives_2D.append(i)
        create_wav_file(audio[i:i+44100], "2D_false_negative_" + str(i))

In [38]:
total = recording_length*30
total_pos_2D = len(positives_2D)
total_neg_2D = total - total_pos_2D
TP_2D = len(true_positives_2D)
FP_2D = len(false_positives_2D)
FN_2D = len(false_negatives_2D)
TN_2D = total_neg_2D - FN_2D 

print("1D Model Metrics:")
print("Total # of clips classified as gunshots: " + str(total_pos_2D))
print("True Positives: " + str(TP_2D))
print("False Positives: " + str(FP_2D))
print("False Negatives: " + str(FN_2D))
print("True Negatives: " + str(TN_2D))

precision_2D = TP_2D / (TP_2D + FP_2D)
recall_2D = TP_2D / (TP_2D + FN_2D)
accuracy_2D = (TP_2D + TN_2D) / total
print("Precision: " + str(precision_2D))
print("Recall: " + str(recall_2D))
print("Accuracy: " + str(accuracy_2D))

1D Model Metrics:
Total # of clips classified as gunshots: 3
True Positives: 3
False Positives: 0
False Negatives: 83
True Negatives: 214
Precision: 1.0
Recall: 0.03488372093023256
Accuracy: 0.7233333333333334
